##Assignment 4 by Ishita Gupta (ID: 49003344)

###Requirements:
>1. [Q5. Test dataloader for CIFAR100](https://colab.research.google.com/drive/158SvaXSUkK_Y5H6HxYam8IAS2HqLTH0G#scrollTo=eKUa9X0o1W5f&line=8&uniqifier=1)
>2. [Q5. Train and test dataloaders for CIFAR 10](https://colab.research.google.com/drive/158SvaXSUkK_Y5H6HxYam8IAS2HqLTH0G#scrollTo=0-wbobPN1W5q&line=2&uniqifier=1)
>3. [Q5. Training from scratch CIFAR 10](https://colab.research.google.com/drive/158SvaXSUkK_Y5H6HxYam8IAS2HqLTH0G#scrollTo=zhXe7gJr1W5x&line=18&uniqifier=1)
>4. [Q5. Test results for CIFAR 10](https://colab.research.google.com/drive/158SvaXSUkK_Y5H6HxYam8IAS2HqLTH0G#scrollTo=zhXe7gJr1W5x&line=18&uniqifier=1)
>5. [Q5. Transfer learning model](https://colab.research.google.com/drive/158SvaXSUkK_Y5H6HxYam8IAS2HqLTH0G#scrollTo=D7h1Ftmc1W53&line=10&uniqifier=1)
>6. [Q5. Test results on transfer model](https://colab.research.google.com/drive/158SvaXSUkK_Y5H6HxYam8IAS2HqLTH0G#scrollTo=Nt34zL3T1W58&line=4&uniqifier=1)


In [0]:
'''
Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun.
Deep Residual Learning for Image Recognition.
In CVPR, 2016.
'''

import torch.nn as nn
import torch.utils.model_zoo as model_zoo

pretrained_settings = {
    "cifar100": {
        'resnet20': 'https://github.com/chenyaofo/CIFAR-pretrained-models/releases/download/resnet/cifar100-resnet20-8412cc70.pth',
        'num_classes': 100
    }
}


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class CifarResNet(nn.Module):

    def __init__(self, block, layers, num_classes=10):
        super(CifarResNet, self).__init__()
        self.inplanes = 16
        self.conv1 = conv3x3(3, 16)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)

        self.layer1 = self._make_layer(block, 16, layers[0])
        self.layer2 = self._make_layer(block, 32, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x
    
def cifar_resnet20_100():
    model = CifarResNet(BasicBlock, [3, 3, 3], num_classes=pretrained_settings['cifar100']['num_classes'])
    model.load_state_dict(model_zoo.load_url(pretrained_settings['cifar100']['resnet20']))
    return model

In [0]:
model_100 = cifar_resnet20_100()
print(model)

Downloading: "https://github.com/chenyaofo/CIFAR-pretrained-models/releases/download/resnet/cifar100-resnet20-8412cc70.pth" to /root/.cache/torch/checkpoints/cifar100-resnet20-8412cc70.pth
100%|██████████| 1.09M/1.09M [00:00<00:00, 7.86MB/s]


NameError: ignored

In [0]:
#implemen the cifar-100 test datalodaer and run the test. 
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)


In [0]:
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_100.parameters(), lr=0.001, momentum=0.9)

In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model_100(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 38 %


In [0]:
#implement the cifar-10 train,val,test dataloader & define the model for cifar-10 using above model

import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)
def cifar_resnet20():
    model = CifarResNet(BasicBlock, [3, 3, 3], num_classes=10)
    return model
    
model = cifar_resnet20()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
#training the cifar-10 from Scratch
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.028
[1,  4000] loss: 1.841
[1,  6000] loss: 1.697
[1,  8000] loss: 1.568
[1, 10000] loss: 1.487
[1, 12000] loss: 1.397
[2,  2000] loss: 1.301
[2,  4000] loss: 1.252
[2,  6000] loss: 1.183
[2,  8000] loss: 1.173
[2, 10000] loss: 1.140
[2, 12000] loss: 1.069
Finished Training


In [0]:
#implement test code & report the accuracy

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 62 %


In [0]:
#implement transfer learning (cifar-100 -> cifar-10)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


[1,  2000] loss: 2.032
[1,  4000] loss: 1.812
[1,  6000] loss: 1.686
[1,  8000] loss: 1.584
[1, 10000] loss: 1.472
[1, 12000] loss: 1.404
[2,  2000] loss: 1.311
[2,  4000] loss: 1.228
[2,  6000] loss: 1.211
[2,  8000] loss: 1.179
[2, 10000] loss: 1.143
[2, 12000] loss: 1.064
Finished Training


In [0]:
#implement test code & report the accuracy

#implement test code & report the accuracy

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 60 %
